In [65]:

import pandas as pd
pd.options.mode.chained_assignment = None
import numpy as np
from openpyxl import load_workbook
import time
import datetime

def forecastImport():
    fc = pd.read_excel("UPDATE.xlsx",sheet_name="FORECAST")
    fc['Item Number'] = fc['Item Number'].astype(str)
    return fc

def forsImport():
    fors = pd.read_excel("UPDATE.xlsx", sheet_name="FORS")
    fors =  fors.rename(columns={"2nd Item Number":"Item Number"})
    fors['Item Number'] = fors['Item Number'].astype(str)
    return fors
    
def orderFileImport():
    orders = pd.read_excel("FILE.xlsx", sheet_name="OPEN_ORDERS")
    orders = pd.DataFrame(orders)
    orders = orders.rename(columns={"Item\nNumber":"Item Number"})
    orders['Item Number'] = orders['Item Number'].astype(str)
    orders['Sales Reporting Code 03'] = orders['Sales Reporting Code 03'].astype(int)
    orders['KEY2'] = orders['Order\nNumber'].astype(str)+orders['Item Number'].astype(str)
    return orders

def accountFileImport():
    acc = pd.read_excel("FILE.xlsx",sheet_name="ACCOUNT")
    acc = acc.fillna(0)
    acc['SOLDTO_ACC'] = acc['SOLDTO_ACC'].astype(int)
    acc['SalesCode'] = acc['SalesCode'].astype(int)
    acc['RBU'] = acc['RBU'].astype(int)
    acc['KEY1'] = acc['RBU'].astype(str)+"_"+acc['SalesCode'].astype(str)+"_"+acc['SOLDTO_ACC'].astype(str)
    acc = acc.drop_duplicates(subset='KEY1',keep='last')
    acc = acc[['GROUP','DIVISION','RBU','SalesCode','SOLDTO_ACC','SOLDTO_NAME','Customer',"KEY1"]]
    return acc

def budgetFileImport(date):
    budget = pd.read_excel("UPDATE.xlsx", sheet_name = "BUDGET")
    bud = budget[['RBU','Address Number','Item Number - Free Format','Sales Category Code 3',date['Month'].iloc[0]]]
    bud = bud.rename(columns = {"Item Number - Free Format":"Item Number",date['Month'].iloc[0]:"Budget"})
    bud[['SOLDTO_ACC', 'SOLDTO_NAME']] = bud['Address Number'].str.split(pat='-', n=1, expand=True)
    bud['SOLDTO_ACC'] = bud['SOLDTO_ACC'].str.strip()
    bud['SOLDTO_ACC'] = bud['SOLDTO_ACC'].str.strip()
    bud['Sales Category Code 3'] = bud['Sales Category Code 3'].replace(' ', np.nan)
    bud['Sales Category Code 3'] = bud['Sales Category Code 3'].astype(float).astype(pd.Int64Dtype())
    bud['KEY1'] = bud['RBU'].astype(str)+"_"+bud['Sales Category Code 3'].astype(str)+"_"+bud['SOLDTO_ACC'].astype(str)
    bud['Item Number'] = bud['Item Number'].astype(str)
    budx = bud
    bud = bud.groupby(['KEY1'])['Budget'].sum()
    bud = pd.DataFrame(bud)
    bud.reset_index(inplace=True)
    
    return bud, budx

def mtdFileImport(con):
    mtd = pd.read_excel("UPDATE.xlsx", sheet_name="MTD")
    
    mtd = mtd.dropna()
    mtd[['SOLDTO_ACC', 'SOLDTO_NAME']] = mtd['Sold\nTo'].str.split(pat='-', n=1, expand=True)
    mtd['SOLDTO_ACC'] = mtd['SOLDTO_ACC'].str.strip()
    mtd['SOLDTO_NAME'] = mtd['SOLDTO_NAME'].str.strip()

    mtd[['RBU', 'RBU_NAME']] = mtd['BU\nHeader'].str.split(pat='-', n=1, expand=True)
    mtd['RBU'] = mtd['RBU'].str.strip()
    mtd['RBU_NAME'] = mtd['RBU_NAME'].str.strip()

    mtd[['SalesCode', 'Code_name']] = mtd['Sales Cat 03'].str.split(pat='-', n=1, expand=True)
    mtd['SalesCode'] = mtd['SalesCode'].str.strip()
    mtd['Code_name'] = mtd['Code_name'].str.strip()

    mtd['RBU'] = mtd['RBU'].astype(int)
    mtd['SalesCode'] = mtd['SalesCode'].astype(int)
    mtd['Branch/\nPlant'] = mtd['Branch/\nPlant'].astype(int)
    mtd['SOLDTO_ACC'] = mtd['SOLDTO_ACC'].astype(int)

    mtd = mtd.rename(columns={"Item\nNumber":"Item Number","Qty\nShipped\nto Date":"QtyShipped"})
    mtd['Item Number'] = mtd['Item Number'].astype(str)
    mtd = pd.merge(mtd, con[["Item Number","Conversion Factor"]], on="Item Number", how='left')
    mtd['Conversion Factor'] = mtd['Conversion Factor'].fillna(1)
    mtd['QtyShipped'] = mtd['Conversion Factor'].astype(int) * mtd['QtyShipped'].astype(int)

    mtd['KEY1'] = mtd['RBU'].astype(str)+"_"+mtd['SalesCode'].astype(str)+"_"+mtd['SOLDTO_ACC'].astype(str)
    
    mtd['KEY2'] = mtd['Order\nNumber'].astype(str)+mtd['Document\nNumber'].astype(str)+mtd['Line\nNumber'].astype(str)
    
    mtd = mtd.drop_duplicates(subset='KEY2', keep='last')
    
    mtd = mtd[(mtd['SalesCode']!=250)|(mtd['SalesCode']!=500)|(mtd['SalesCode']!=110)]
    
    return mtd

def otherFileImport():
    date = pd.read_excel("File.xlsx",sheet_name="ORDERDATE")
    con = pd.read_excel("FILE.xlsx",sheet_name="CONVERSION")
    return date, con

    
    
def importProcess():
    try:
        print("1")
        date, con = otherFileImport()
        print("2")
        orders = orderFileImport()
        print("3")
        acc = accountFileImport()
        print("4")
        mtd = mtdFileImport(con)
        bud, budx = budgetFileImport(date)
        fors = forsImport()
        print("5")
        fc = forecastImport()
        results = True

        
    except:
        print("File & cleaning error, please check your files")
        orders = []
        acc = []
        mtd =[]
        date =[]
        coom =[]
        budx = []
        fc = []
        results = False
    return orders, acc, mtd, date, con, results, budx, fc, bud, fors
    
    
def concatProcess_sales(mtd,acc,bud):
    print("WHYY")
    custSales = mtd.groupby(['KEY1'])['QtyShipped'].sum()
    custSales = pd.DataFrame(custSales)
    custSales.reset_index(inplace=True)
    print("   7.1")
    
    acc = pd.merge(acc, bud, on="KEY1",how='left')
    UnRegisteredBudget = bud[~bud['KEY1'].isin(acc['KEY1'])]
    print("   7.2")
    
    acc = pd.merge(acc,custSales, on="KEY1", how='left')
    acc['QtyShipped'] = acc['QtyShipped'].fillna(0)
    print("   7.3")
    
    UnRegistered = custSales[~custSales['KEY1'].isin(acc['KEY1'])]
    UnRegistered = pd.merge(UnRegistered,mtd[['KEY1','Ship\nTo','SOLDTO_ACC', 'SOLDTO_NAME', 'RBU', 'RBU_NAME', 'SalesCode']], 
                       on="KEY1", how="left")
    print("   7.4")
    
    acc = acc.drop_duplicates(subset='KEY1', keep="last")
    
    return acc, UnRegistered, UnRegisteredBudget

def concatProcess_orders(acc,orders,con):
    orders = pd.merge(orders,con[["Item Number","Conversion Factor"]], on="Item Number", how='left')
    orders = orders.drop_duplicates(subset="KEY2", keep='last')
    orders['Conversion Factor'] = orders['Conversion Factor'].fillna(1)
    
    orders['TotalOpenOrders'] = orders['Qty\nOrder/\nTransaction'].astype(int)*orders['Conversion Factor'].astype(int)
    orders['TotalBackOrders'] = orders['Qty\nBackorder/\nHeld'].astype(int)*orders['Conversion Factor'].astype(int)
    
    def pastDue_Orders(row):
        if row['Requested\nDate'] < date['Beg_Month'].iloc[0]:
            return row['TotalOpenOrders']
        else:
            return 0
    
    def currDue_Orders(row):
        if row['Requested\nDate'] > date['Beg_Month'].iloc[0] and row['Requested\nDate'] < date['End_Month'].iloc[0]:
            return row['TotalOpenOrders']
        else:
            return 0

    def postDue_Orders(row):
        if row['Requested\nDate'] > date['End_Month'].iloc[0]:
            return row['TotalOpenOrders']
        else:
            return 0

    # Apply the custom_condition function to create a new column 'C' based on the condition
    orders['PastDue_Orders'] = orders.apply(pastDue_Orders, axis=1)
    orders['CurrDue_Orders'] = orders.apply(currDue_Orders, axis=1)
    orders['PostDue_Orders'] = orders.apply(postDue_Orders, axis=1)
    
    def pastDue_bo(row):
        if row['Requested\nDate'] < date['Beg_Month'].iloc[0]:
            return row['TotalBackOrders']
        else:
            return 0
    
    def currDue_bo(row):
        if row['Requested\nDate'] > date['Beg_Month'].iloc[0] and row['Requested\nDate'] < date['End_Month'].iloc[0]:
            return row['TotalBackOrders']
        else:
            return 0

    def postDue_bo(row):
        if row['Requested\nDate'] > date['End_Month'].iloc[0]:
            return row['TotalBackOrders']
        else:
            return 0

    # Apply the custom_condition function to create a new column 'C' based on the condition
    orders['PastDue_BO'] = orders.apply(pastDue_bo, axis=1)
    orders['CurrDue_BO'] = orders.apply(currDue_bo, axis=1)
    orders['PostDue_BO'] = orders.apply(postDue_bo, axis=1)

    orders['KEY1'] = orders['BU\nHeader'].astype(str)+"_"+orders['Sales Reporting Code 03'].astype(str)+"_"+orders['Sold\nTo'].astype(str)
    
    grp = orders.groupby(['KEY1'])[['TotalOpenOrders','TotalBackOrders',"PastDue_Orders","CurrDue_Orders","PostDue_Orders",
                               "PastDue_BO","CurrDue_BO","PostDue_BO"]].sum()
    grp = pd.DataFrame(grp)
    grp.reset_index(inplace=True)
    acc = pd.merge(acc,grp[['KEY1','TotalOpenOrders',"TotalBackOrders",
                           'PastDue_Orders','CurrDue_Orders','PostDue_Orders',
                          "PastDue_BO",'CurrDue_BO','PostDue_BO']], on='KEY1',how='left')
    
    UnRegisteredOrders = orders[~orders['KEY1'].isin(acc['KEY1'])]
    
    acc = acc.drop_duplicates(subset='KEY1', keep="last")
    
    return acc, UnRegisteredOrders, orders
    
orders, acc, mtd, date, con, results, budx, fc, bud, fors = importProcess()
if results == False:
    print("Error, Dash Data preparation Failed")
    result = False
else:
    print('6')
    try:
        acc, UnRegistered, UnRegisteredBudget = concatProcess_sales(mtd,acc,bud)
        print("7")
        acc, UnRegisteredOrders, orders = concatProcess_orders(acc,orders,con)
        print("8")
        result = True
    except:
        print("Files concatination failed")
        result = False
        
        
#acc, UnRegistered, UnRegisteredBudget = concatProcess_sales(mtd,acc,bud)
#acc, UnRegisteredOrders, orders = concatProcess_orders(acc,orders,con)
def itemProcess(budx,mtd,orders,fc,acc,fors):
    budx = budx.rename(columns={'Sales Category Code 3':"SalesCode"})
    budx['KEY3'] = budx['RBU'].astype(str)+"_"+budx['SalesCode'].astype(str)+"_"+budx['Item Number'].astype(str)
    budx = budx.groupby(['KEY3','RBU','SalesCode','Item Number'])['Budget'].sum()
    budx = pd.DataFrame(budx)
    budx.reset_index(inplace=True)
    
    mtd['KEY3'] = mtd['RBU'].astype(str)+"_"+mtd['SalesCode'].astype(str)+"_"+mtd['Item Number'].astype(str)
    mtd = mtd[['KEY3','RBU',"SalesCode","Item Number","QtyShipped"]]
    mtd = mtd.groupby(['KEY3','RBU','SalesCode','Item Number'])['QtyShipped'].sum()
    mtd = pd.DataFrame(mtd)
    mtd.reset_index(inplace=True)
    
    fc = pd.merge(fc,fors[['Item Number','Sls Cd3']],on='Item Number',how='left')
    fc['Sls Cd3'] = fc['Sls Cd3'].replace(' ', np.nan)
    fc['Sls Cd3'] = fc['Sls Cd3'].astype(float).astype(pd.Int64Dtype())
    fc = fc.rename(columns={"Sls Cd3":"SalesCode"})
    fc['KEY3'] = fc['RBU'].astype(str)+"_"+fc['SalesCode'].astype(str)+"_"+fc['Item Number'].astype(str)
    fc = fc.groupby(['KEY3', 'RBU',"SalesCode", 'Item Number']).apply(lambda x: x.iloc[:, 2:5].sum())
    fc.reset_index(inplace=True)
    
    test = pd.merge(budx,fc, on=['KEY3','RBU',"SalesCode",'Item Number'], how='outer')
    test = test.fillna(0)
    
    test = pd.merge(test,mtd,on=['KEY3','RBU',"SalesCode",'Item Number'], how="outer")
    test = test.fillna(0)
    
    orders = orders.rename(columns={'Sales Reporting Code 03':"SalesCode"})
    orders['SalesCode'] = orders['SalesCode'].replace(' ', np.nan)
    orders['SalesCode'] = orders['SalesCode'].astype(float).astype(pd.Int64Dtype())
    
    ordersx = orders[['BU\nHeader',"SalesCode","Item Number",'TotalOpenOrders',"TotalBackOrders",
                  'PastDue_Orders','CurrDue_Orders','PostDue_Orders',
                  "PastDue_BO",'CurrDue_BO','PostDue_BO']]
    ordersx = ordersx.rename(columns = {'BU\nHeader':"RBU"})
    ordersx['KEY3'] = ordersx['RBU'].astype(str)+"_"+ordersx['SalesCode'].astype(str)+"_"+ordersx['Item Number'].astype(str)
    ordersx = ordersx.groupby(['KEY3', 'RBU',"SalesCode",'Item Number'])[['TotalOpenOrders', 'TotalBackOrders',
                                                       'PastDue_Orders', 'CurrDue_Orders',
                                                       'PostDue_Orders', 'PastDue_BO',
                                                       'CurrDue_BO', 'PostDue_BO']].sum()
    ordersx.reset_index(inplace=True)
    
    test = pd.merge(test,ordersx,on=['KEY3','RBU',"SalesCode",'Item Number'], how="outer")
    test = test.fillna(0)
    test = test.drop_duplicates(subset='KEY3', keep='last')
    
    rbu = acc[['GROUP','DIVISION','RBU','SalesCode']]
    rbu = rbu.drop_duplicates(subset=['GROUP','DIVISION','RBU','SalesCode'], keep='last')
    rbu['KEY4'] = rbu['RBU'].astype(str)+"_"+rbu['SalesCode'].astype(str)
    
    #test = pd.merge(test, fors[['Item Number',"Sls Cd3"]], on="Item Number", how='left')
    ##test['Sls Cd3'] = test['Sls Cd3'].replace(' ', np.nan)
    #test['Sls Cd3'] = test['Sls Cd3'].astype(float).astype(pd.Int64Dtype())
    test['KEY4'] = test['RBU'].astype(str)+"_"+test['SalesCode'].astype(str)
    
    test = pd.merge(test,rbu[['KEY4','GROUP','DIVISION']], on='KEY4',how='left')
    test = test.drop_duplicates(subset='KEY3', keep='last')
    
    return test

test = itemProcess(budx,mtd,orders,fc,acc,fors)

def publishExcelFile(acc,UnRegistered, UnRegisteredOrders,test):
    excel_file = 'DashFile.xlsx'
    book = load_workbook(excel_file)

    # Create a Pandas ExcelWriter using the loaded workbook
    with pd.ExcelWriter(excel_file, engine='openpyxl', mode='a',if_sheet_exists='replace') as writer:
        writer.book = book

        # Check if the 'OpenOrders' sheet exists, and if so, clear its content
        if 'CustomerSales' in writer.book.sheetnames:
            sheet = writer.book['CustomerSales']
            sheet.delete_rows(sheet.min_row + 1, sheet.max_row)

        # Write the DataFrame to the 'OpenOrders' sheet
        acc.to_excel(writer, sheet_name='CustomerSales', index=False)
        
        if "DivisionSalesData" in writer.book.sheetnames:
            sheet = writer.book['DivisionSalesData']
            sheet.delete_rows(sheet.min_row + 1, sheet.max_row)
            
        test.to_excel(writer, sheet_name="DivisionSalesData", index=False)

        # Check if the 'Shipped' sheet exists, and if so, clear its content
        if 'UnregisteredSales' in writer.book.sheetnames:
            sheet = writer.book['UnregisteredSales']
            sheet.delete_rows(sheet.min_row + 1, sheet.max_row)

        UnRegistered.to_excel(writer, sheet_name='UnregisteredSales', index=False)

        # Check if the 'Update' sheet exists, and if so, clear its content
        if 'UnregisteredOrders' in writer.book.sheetnames:
            sheet = writer.book['UnregisteredOrders']
            sheet.delete_rows(sheet.min_row + 1, sheet.max_row)

        UnRegisteredOrders.to_excel(writer, sheet_name='UnregisteredOrders', index=False)

    # Save the changes to the Excel file
    book.save(excel_file)
    
    
if result == False:
    print("Unsuccessfull")
else:
    publishExcelFile(acc,UnRegistered, UnRegisteredOrders,test)
    print("Successful")
    time.sleep(4)

    

1
2
3
4
5
6
WHYY
   7.1
   7.2
   7.3
   7.4
7
8


C:\Users\gwalal\AppData\Local\Temp\ipykernel_25856\2452703155.py:312: FutureWarning: Setting the `book` attribute is not part of the public API, usage can give unexpected or corrupted results and will be removed in a future version
  writer.book = book


Successful


In [43]:
wo = pd.read_excel("UPDATE.xlsx",sheet_name="WORKORDER")

In [44]:
def wo_prep(wo):
    wo['[SCHEDULED DATE]'] = pd.to_datetime(wo['[SCHEDULED DATE]'], format='%d-%m-%Y')
    wo['Year'] = wo['[SCHEDULED DATE]'].dt.year
    wo['Year'] = wo['Year'].astype(int)
    wo['Month'] = wo['[SCHEDULED DATE]'].dt.month
    wo['Month'] = wo['Month'].astype(int)
    wo['weekofyear'] = wo['[SCHEDULED DATE]'].dt.isocalendar().week
    wo['weekofyear'] = wo['weekofyear'].astype(int)
    wo['Today'] = date.today()
    wo['TodayW'] = wo['Today'].apply(lambda x:x.isocalendar()[1])
    wo['TodayY'] = pd.to_datetime(wo['Today'])
    wo['TodayY'] =  wo['TodayY'].dt.year

    conditions = [
        (wo['Year'] < wo['TodayY']),
        (wo['Year'] == wo['TodayY']) & (wo['weekofyear'] < wo['TodayW']),
        (wo['Year'] == wo['TodayY']) & (wo['weekofyear'] == wo['TodayW']),
        (wo['Year'] == wo['TodayY']) & (wo['weekofyear'] == wo['TodayW']+1),
        (wo['Year'] == wo['TodayY']) & (wo['weekofyear'] == wo['TodayW']+2),
        (wo['Year'] == wo['TodayY']) & (wo['weekofyear'] == wo['TodayW']+3),
        (wo['Year'] == wo['TodayY']) & (wo['weekofyear'] == wo['TodayW']+4),
        (wo['Year'] == wo['TodayY']) & (wo['weekofyear'] > wo['TodayW']+4),
        (wo['Year'] > wo['TodayY']) & (wo['weekofyear'] == wo['TodayW']+1),
        (wo['Year'] > wo['TodayY']) & (wo['weekofyear'] == wo['TodayW']+2),
        (wo['Year'] > wo['TodayY']) & (wo['weekofyear'] == wo['TodayW']+3),
        (wo['Year'] > wo['TodayY']) & (wo['weekofyear'] == wo['TodayW']+4),
        (wo['Year'] > wo['TodayY']) & (wo['weekofyear'] > wo['TodayW']+4)]

    values = ['ByPass', 'ByPass', 'Current', 'Curr+1','Curr+2','Curr+3','Curr+4','Curr+5','Curr+1','Curr+2','Curr+3','Curr+4','Curr+5']

    
    wo['WO_Status'] = np.select(conditions, values)

    cols = ['ByPass', 'Current', 'Curr+1','Curr+2','Curr+3','Curr+4','Curr+5']

    for i in range(7):

        condition = [wo['WO_Status']== cols[i]]
        val = [wo['[QTY OUTSTANDING]']]

        wo[cols[i]] = np.select(condition,val)
        
    wo = wo.fillna(0)

    wo.rename(columns = {"ITEM NUMBER":"ItemNumber"}, inplace = True)
    wo['ItemNumber'] = wo['ItemNumber'].astype(str)
    wo = wo[["ItemNumber",'ByPass', 'Current','Curr+1','Curr+2','Curr+3','Curr+4','Curr+5']]
    wo = wo.pivot_table(index = "ItemNumber", aggfunc= "sum")
    wo = wo.reset_index()
    column_order = ["ItemNumber", 'ByPass', 'Current', 'Curr+1', 'Curr+2', 'Curr+3', 'Curr+4', 'Curr+5']
    wo = wo[column_order]
    
    
    
    current_date = datetime.now()
    days_until_friday = (4 - current_date.weekday() + 7) % 7
    fridays = []
    for _ in range(8):
        fridays.append("WO " + (current_date + timedelta(days=days_until_friday)).strftime("%d-%b"))
        days_until_friday += 7
    
    wo = wo.rename(columns={"Curr+1":fridays[1],"Curr+2":fridays[2],
                       "Curr+3":fridays[3],"Curr+4":fridays[5],
                       "Curr+5":"POSTDUE",})
    return wo

In [45]:
wo = wo_prep(wo)

In [60]:
final = test.groupby(['Item Number',"SalesCode"]).apply(lambda x: x.iloc[:, 4:17].sum())

In [61]:
final.reset_index(inplace=True)

In [63]:
final

,Item Number,SalesCode,Budget,FC_Sep,FC_Oct,FC_Nov,QtyShipped,TotalOpenOrders,TotalBackOrders,PastDue_Orders,CurrDue_Orders,PostDue_Orders,PastDue_BO,CurrDue_BO,PostDue_BO
0,01030,110,0.0,0.0,0.0,0.0,6.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,01830,110,0.0,0.0,0.0,0.0,46.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,02/1331MOR,250,0.0,0.0,0.0,0.0,1000.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,02/1604MOR,250,0.0,0.0,0.0,0.0,2000.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,02/1605-1MOR,250,0.0,0.0,0.0,0.0,5000.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
7650,Z95MFR,810,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
7651,Z96,100,14.0,15.0,16.0,21.0,13.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
7652,Z96BOS,300,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
7653,Z96FR,90,2.0,6.0,9.0,10.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [59]:
test.to_excel("test.xlsx")